In [4]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

In [5]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

classes = [0,1,2]
classes_names = ['GALAXY','STAR','QSO']

y_train = df_train["class"].values
X_train = df_train.drop(["class"], axis=1).values

# Preparar los datos del training
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
del(X_train)

# GridSearch automaticamente va a dividir en trainig/validation internamente
# usando una estrategia llamada K-Folds
# X_train, X_val, y_train, y_val = train_test_split(X_train_scaled, y_train, test_size = 0.33, random_state = 42)

X_train = X_train_scaled

y_test = df_test["class"].values
X_test = df_test.drop(["class"], axis=1).values

# liberar de memoria
del(df_train)
del(df_test)

In [17]:
# Algo util es poder desordenar los datos
from sklearn.utils import shuffle

# Shuffle X_train and y_train together
X_train, y_train = shuffle(X_train, y_train, random_state=42)


In [6]:
print("shapes")
print("X_train", X_train.shape)
print("y_train", y_train.shape)

print("X_val", X_test.shape)
print("y_val", y_test.shape)

shapes
X_train (121422, 9)
y_train (121422,)
X_val (16949, 9)
y_val (16949,)


# Como seleccionar Hiperparametros
Esta es la parte que requiere mayor poder de computo, estan las ideas de hiperparametros que son cantidades que decidimos al construir los modelos, en el caso de los arboles de decisión es la cantidad de nodos maximos

Existen varios metodos para seleccionarlos, en este caso enseñaré el metodo más facil de entender y que solo requiere un montón de computo.
- Grid Search

Pueds probar alternativas más sofisticadas como
- Bayesian Hyperparameter Tunning; que he utilizado para entrenar redes neuronales y encontrar las versiones más eficientes

In [18]:
# Verifica el número de clases en el conjunto de entrenamiento
print("Número de clases en y_train:", np.unique(y_train[:100]))

# Verifica el número de clases en el conjunto de validación
print("Número de clases en y_val:", np.unique(y_test[:100]))


Número de clases en y_train: [0 1 2]
Número de clases en y_val: [0 1 2]


In [ ]:
# Halving Grid Search es experimental, asi que requiere importar esto extra
# from sklearn.experimental import enable_halving_search_cv 
# from sklearn.model_selection import HalvingGridSearchCV

from sklearn.model_selection import GridSearchCV

# Definir el clasificador base
base_clf = RandomForestClassifier()

# Definir el clasificador de AdaBoost
boosted_clf = AdaBoostClassifier(estimator=base_clf, algorithm="SAMME")

# Definir los hiperparámetros que queremos ajustar
param_grid = {
    'estimator__n_estimators': [10, 50, 100, 200],  # Número de árboles en el clasificador base (Random Forest)
    'n_estimators': [50, 100, 150, 200],  # Número de iteraciones de AdaBoost
    'learning_rate': [0.001, 0.01, 0.1, 1.0],  # Tasa de aprendizaje de AdaBoost
    'estimator__max_depth': [5, 10, 20],  # Profundidad máxima del Random Forest
    'estimator__min_samples_split': [2, 5, 10, 50]  # Mínimo de muestras por nodo para dividir
}

# Usar GridSearchCV para ajustar los hiperparámetros con barra de progreso
# grid_search = HalvingGridSearchCV(boosted_clf, param_grid, scoring='accuracy', cv=3, n_jobs=1, verbose=3)
grid_search = GridSearchCV(boosted_clf, param_grid, scoring='accuracy', cv=3, n_jobs=1, verbose=10)


grid_search.fit(X_train, y_train)


# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

In [ ]:
# Evaluar las predicciones en el conjunto de validación
y_pred_boosted = grid_search.predict(X_val)

# Mostrar el informe de clasificación
print(classification_report(y_val, y_pred_boosted, target_names=["galaxy", "star", "qso"]))

# Mostrar la matriz de confusión
ConfusionMatrixDisplay.from_estimator(grid_search.best_estimator_, X_val, y_val, display_labels=["galaxy", "star", "qso"], cmap='Blues')
plt.show()